In [7]:
#Mounting google drive 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
#To ignore the warnings
import warnings
warnings.filterwarnings("ignore")
 

In [9]:
#Import basic python librairies for image preprocessing
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.pywrap_tensorflow_internal import *
import keras
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten
from keras.models import Sequential
from keras.preprocessing import image
from PIL import Image

In [10]:
#training image data generator 
#---try to generate image---
from keras.preprocessing import image
train_datagen = image.ImageDataGenerator(rescale=1/255,horizontal_flip =True, zoom_range=0.2)
train_data=train_datagen.flow_from_directory(directory='/content/drive/MyDrive/covid-19_detection_dataset/training', target_size =(500,550), batch_size=6, class_mode ='binary')
                                     

Found 572 images belonging to 2 classes.


In [11]:
 # class indication
train_data.class_indices                                    

{'Covid': 0, 'Normal': 1}

In [12]:
#Validation image data
# batch size is the number of samples that will be passed through to the network at one time.  
val_datagen = image.ImageDataGenerator(rescale=1/255)
val_data=train_datagen.flow_from_directory(directory='/content/drive/MyDrive/covid-19_detection_dataset/validitation',target_size =(500,550),batch_size=4, class_mode ='binary')

Found 572 images belonging to 2 classes.


In [13]:
#Build CNN model (input layer)
#rectified linear activation function or ReLU for short is a piecewise linear function 
#that will output the input directly if it is positive, otherwise, it will output zero
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten
model=Sequential()
model.add(Conv2D(filters=32, kernel_size=(3,3),activation='relu',input_shape=(500,550,3)))
model.add(Conv2D(filters = 64, kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D())
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D())
model.add(Dropout(0.25))


model.add(Conv2D(filters = 128, kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D())
model.add(Dropout(0.25))
model.add(Flatten())
model.add( Dense(64,activation='relu'))

In [14]:
#Doutput layer
model.add(Dropout(0.25))
model.add( Dense(1,activation='sigmoid'))

In [15]:
#model compilation
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
#model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 498, 548, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 496, 546, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 248, 273, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 248, 273, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 246, 271, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 123, 135, 64)     0         
 2D)                                                    

Plotting Graphs

In [ ]:
#model fit generator
#One epoch is when the entire dataset is passed forward and backward through the neural network once
# An epoch means training the neural network with all the training data for one cycle.
history = model.fit_generator(train_data, steps_per_epoch=4,epochs=50,validation_data=val_data)

accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'r', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.ylabel('accuracy') 
plt.xlabel('epoch')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.ylabel('loss') 
plt.xlabel('epoch')
plt.legend()
plt.show()

Epoch 1/50
4/4 [==============================] - 351s 114s/step - loss: 5.0416 - accuracy: 0.7083 - val_loss: 0.7791 - val_accuracy: 0.5210
Epoch 2/50
4/4 [==============================] - 194s 62s/step - loss: 0.9596 - accuracy: 0.5417 - val_loss: 0.6906 - val_accuracy: 0.4790
Epoch 3/50
4/4 [==============================] - 195s 62s/step - loss: 0.7864 - accuracy: 0.4583 - val_loss: 0.6911 - val_accuracy: 0.5210
Epoch 4/50
4/4 [==============================] - 193s 62s/step - loss: 0.6845 - accuracy: 0.5417 - val_loss: 0.6905 - val_accuracy: 0.7238
Epoch 5/50
4/4 [==============================] - 189s 61s/step - loss: 0.6856 - accuracy: 0.6000 - val_loss: 0.6701 - val_accuracy: 0.8094
Epoch 6/50
4/4 [==============================] - 197s 63s/step - loss: 0.6854 - accuracy: 0.6250 - val_loss: 0.6493 - val_accuracy: 0.8444
Epoch 7/50
4/4 [==============================] - 197s 63s/step - loss: 0.6234 - accuracy: 0.8333 - val_loss: 0.6391 - val_accuracy: 0.7815
Epoch 8/50
4/4 [===

In [ ]:
#!pip install Pillow==5.3.0
from PIL import Image

In [ ]:
#checking how model is performing
path="/content/drive/MyDrive/covid-19_detection_dataset/validitation/Covid/covid_8.jpeg"
img=Image.open(path)
img=image.img_to_array(img)/255
plt.imshow(img)
img.resize(500,550,3)
img=np.array([img])
img.shape

In [ ]:
#model prediction
model.predict(img)
